In [ ]:
import os
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
np.random.seed(9630)
tf.random.set_seed(9630)

print(tf.__version__)

2.7.0


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
size_input = 784
size_hidden = [128, 64]
size_output = 10
number_of_train_examples = 60000
number_of_test_examples = 10000

In [ ]:
# load datasets
mnist = tf.keras.datasets.mnist
fashion_mnist = tf.keras.datasets.fashion_mnist

In [ ]:
def mnist_normalize():
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
  X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return (X_train, y_train), (X_test, y_test)

def fashion_normalize():
  (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
  X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
  X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return (X_train, y_train), (X_test, y_test)

In [ ]:
# Split dataset into batches
# (X_train, y_train), (X_test, y_test) = mnist_normalize()
(X_train, y_train), (X_test, y_test) = fashion_normalize()

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)

In [ ]:
# Define class to build MLP model
class MLP(object):
  def __init__(self, size_input, size_hidden, size_output, device = None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer (2)
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. 
    If None, the device will automatically decide during Eager Execution ('gpu')
    """

    self.size_input, self.size_hidden, self.size_output, self.device =\
    size_input, size_hidden, size_output, device

    # Initialize weights of input layer
    self.W0 = tf.Variable(tf.random.normal([self.size_input, self.size_input]))
    # Initialize biases for input layer
    self.b0 = tf.Variable(tf.random.normal([1, self.size_input]))
    # Initialize weights between input layer and hidden layer 1
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden[0]]))
    # Initialize biases for hidden layer 1
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden[0]]))
    # Initialize weights between hidden layer 1 and hidden layer 2
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden[0], self.size_hidden[1]]))
    # Initialize biases for hidden layer 2
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden[1]]))
    # Initialize weights between hidden layer 2 and output layer
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden[1], self.size_output]))
    # Initialize biases for output layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))

    # Define variables to be updated during backpropagation
    self.variables = [self.W0, self.W1, self.W2, self.W3, self.b0, self.b1, self.b2, self.b3]

  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """

    if self.device is not None:
      with tf.device('gpu:0' if self.device == 'gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
    return self.y

  def loss(self, y_pred, y_true):
    """
    y_pred: Tensor of shape (batch_size, size_output)
    y_true: Tensor of shape (batch_size, size_output)
    """

    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)

  def loss_with_regularization(self, y_pred, y_true, lambd = 0.7):
    m = y_true.shape[1]
    W0 = self.variables[0]
    W1 = self.variables[1]
    W2 = self.variables[2]
    W3 = self.variables[3]

    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      pre_loss = self.loss(predicted, y_train)
    L2_regularization_loss = lambd * (np.sum(np.square(W0)) + np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3))) / (2 * m)
    final_loss = pre_loss + L2_regularization_loss

    return final_loss

  def backward_with_regularization(self, X_train, y_train):
    """
    backward pass
    """

    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss_with_regularization(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))

  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """

    # Cast X to float 32
    X_tf = tf.cast(X, dtype=tf.float32)
    # Remember to normalize your dataset before moving forward

    # Compute values in input layer
    # what0 = tf.matmul(X_tf, self.W0) + self.b0
    # hhat0 = tf.nn.relu(what0)

    # Compute values in hidden layer 1
    what1 = tf.matmul(X_tf, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)

    # Compute values in hidden layer 2
    what2 = tf.matmul(hhat1, self.W2) + self.b2
    hhat2 = tf.nn.relu(what2)

    # Compute output
    output = tf.matmul(hhat2, self.W3) + self.b3
    output = tf.nn.softmax(output)

    # Now consider two things
    # First, look at inbuild loss functions if they work with softmax and then change this
    # Second, add tf.Softmax(output) and then return this variable
    return output

#Train Model

In [ ]:
# Set number of epochs
NUM_EPOCHS = 10

In [ ]:
def compute_acc(y_pred, y_true):
  maxid = lambda x: np.argmax(x)
  y_pred_max = np.array([maxid(i) for i in y_pred])
  y_true_max = np.array([maxid(j) for j in y_true])
  acc = sum(y_pred_max == y_true_max) / len(y_pred_max)
  return acc

In [ ]:
def compute_std(y_pred):
  y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
  std_dev = np.std(y_pred_tf)
  std_err = std_dev / np.sqrt(len(y_pred_tf))
  variance = std_dev**2
  return std_err, variance

In [ ]:
# Initialize model using GPU
mlp_on_gpu = MLP(size_input, size_hidden, size_output, device = 'gpu')
time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  acc = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(9630)).batch(40)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total = loss_total + mlp_on_gpu.loss_with_regularization(preds, outputs)
    mlp_on_gpu.backward_with_regularization(inputs, outputs)
    acc = acc + compute_acc(preds, outputs)
  print('Number of Epoch: {} - Average MSE: {}' .format(epoch + 1, np.sum(loss_total) / X_train.shape[0]))
  print('                   - Accuracy: {}' .format(acc/len(train_ds)))
 
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
# For per epoch_time
# epoch_time = Total time / Number of epochs

Number of Epoch: 1 - Average MSE: 5749763.822933333
                   - Accuracy: 0.49508333333333376
Number of Epoch: 2 - Average MSE: 5774463.249066667
                   - Accuracy: 0.8080333333333333
Number of Epoch: 3 - Average MSE: 5803651.345066667
                   - Accuracy: 0.8824166666666676
Number of Epoch: 4 - Average MSE: 5835331.447466667
                   - Accuracy: 0.9063499999999999
Number of Epoch: 5 - Average MSE: 5865088.0682666665
                   - Accuracy: 0.917450000000001
Number of Epoch: 6 - Average MSE: 5891383.842133333
                   - Accuracy: 0.92505
Number of Epoch: 7 - Average MSE: 5914759.9872
                   - Accuracy: 0.9304999999999995
Number of Epoch: 8 - Average MSE: 5934680.200533333
                   - Accuracy: 0.9352500000000009
Number of Epoch: 9 - Average MSE: 5948664.490666667
                   - Accuracy: 0.9393000000000012
Number of Epoch: 10 - Average MSE: 5956097.365333334
                   - Accuracy: 0.94243333333

#Test Model

In [ ]:
test_loss_total = tf.Variable(0, dtype=tf.float32)

# test_loss_total = 0.0
test_acc = 0
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  # b = mlp_on_gpu.loss(preds, outputs)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
  test_acc = test_acc + compute_acc(preds, outputs)
  err, var = compute_std(preds)
  
# a = (test_loss_total.numpy() / X_train.shape[0])
# print(X_train.shape[0])
# print(test_loss_total.numpy())
# print(b)
print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_test.shape[0]))
print('Accuracy: {}' .format(test_acc/len(test_ds)))
print('Standard Error: {}' .format(err))
print('Variance: {}' .format(var))

Test MSE: 0.0260
Accuracy: 0.8694


# 10 Trials - Fashion MNIST

In [ ]:
mse_fashion = []
accuracy_fashion = []
err_fashion = []
var_fashion = []
for i in range(10):
  test_loss_total = tf.zeros([1,1], dtype=tf.float32)
  test_acc = 0
  test_err = 0
  test_var = 0
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(25).batch(40)
  for inputs, outputs in test_ds:
    preds = mlp_on_gpu.forward(inputs)
    test_loss_total = test_loss_total + mlp_on_gpu.loss_with_regularization(preds, outputs)
    mlp_on_gpu.backward_with_regularization(inputs, outputs)
    test_acc = test_acc + compute_acc(preds, outputs)
    test_err = test_err + compute_std(preds)[0]
    test_var = test_var + compute_std(preds)[1]
  mse_fashion.append(np.sum(test_loss_total) / X_test.shape[0])
  accuracy_fashion.append(test_acc/len(test_ds))
  err_fashion.append(test_err/len(test_ds))
  var_fashion.append(test_var/len(test_ds))
  print('Number of Trial: {} - Average MSE: {}' .format(i + 1, np.sum(test_loss_total) / X_test.shape[0]))
  print('                   - Accuracy: {}' .format(test_acc/len(test_ds)))
  print('                   - Standard Error: {}' .format(test_err/len(test_ds)))
  print('                   - Variance: {}' .format(test_var/len(test_ds)))
 
print(mse_fashion)
print(accuracy_fashion)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Trial: 1 - Average MSE: 5957405.0816
                   - Accuracy: 0.8669000000000009
Number of Trial: 2 - Average MSE: 5957627.0848
                   - Accuracy: 0.8683999999999998
Number of Trial: 3 - Average MSE: 5957738.9056
                   - Accuracy: 0.8690999999999998
Number of Trial: 4 - Average MSE: 5957738.9056
                   - Accuracy: 0.8674000000000001
Number of Trial: 5 - Average MSE: 5957735.2192
                   - Accuracy: 0.8683999999999998
Number of Trial: 6 - Average MSE: 5957732.7616
                   - Accuracy: 0.8682000000000005
Number of Trial: 7 - Average MSE: 5957756.5184
                   - Accuracy: 0.8676
Number of Trial: 8 - Average MSE: 5958223.0528
                   - Accuracy: 0.8684
Number of Trial: 9 - Average MSE: 5959062.7328
                   - Accuracy: 0.8693999999999994
Number of Trial: 10 - Average MSE: 5959101.6448
                   - Accuracy: 0.8682999999999998
[5957405.0816, 5957627.0848, 5957738.9056, 5957738.90

In [1]:
# print(np.mean(accuracy_fashion))
# print(np.var(accuracy_fashion))
# plt.plot(accuracy_fashion, label = 'mean:0.87, var:5.06e-07')
# plt.legend(loc='upper right', bbox_to_anchor = (1,1.25))
# plt.xlabel('Accuracy')
# plt.ylabel('Count')
# plt.title('L2 Regularization - Fashion MNIST')
# plt.show()

# 10 Trials - MNIST

In [ ]:
mse_mnist = []
accuracy_mnist = []
err_mnist = []
var_mnist = []
for i in range(10):
  test_loss_total = tf.zeros([1,1], dtype=tf.float32)
  test_acc = 0
  test_err = 0
  test_var = 0
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(25).batch(40)
  for inputs, outputs in test_ds:
    preds = mlp_on_gpu.forward(inputs)
    test_loss_total = test_loss_total + mlp_on_gpu.loss_with_regularization(preds, outputs)
    mlp_on_gpu.backward_with_regularization(inputs, outputs)
    test_acc = test_acc + compute_acc(preds, outputs)
    test_err = test_err + compute_std(preds)[0]
    test_var = test_var + compute_std(preds)[1]
  mse_mnist.append(np.sum(test_loss_total) / X_test.shape[0])
  accuracy_mnist.append(test_acc/len(test_ds))
  err_mnist.append(test_err/len(test_ds))
  var_mnist.append(test_var/len(test_ds))
  print('Number of Trial: {} - Average MSE: {}' .format(i + 1, np.sum(test_loss_total) / X_test.shape[0]))
  print('                   - Accuracy: {}' .format(test_acc/len(test_ds)))
  print('                   - Standard Error: {}' .format(test_err/len(test_ds)))
  print('                   - Variance: {}' .format(test_var/len(test_ds)))
 
print(mse_mnist)
print(accuracy_mnist)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Trial: 1 - Average MSE: 5763265.3312
                   - Accuracy: 0.9486999999999988
Number of Trial: 2 - Average MSE: 5763272.704
                   - Accuracy: 0.9481999999999992
Number of Trial: 3 - Average MSE: 5763290.3168
                   - Accuracy: 0.9483999999999987
Number of Trial: 4 - Average MSE: 5763291.5456
                   - Accuracy: 0.9498999999999984
Number of Trial: 5 - Average MSE: 5763296.8704
                   - Accuracy: 0.9481999999999989
Number of Trial: 6 - Average MSE: 5763310.7968
                   - Accuracy: 0.9484999999999992
Number of Trial: 7 - Average MSE: 5763328.4096
                   - Accuracy: 0.9492999999999991
Number of Trial: 8 - Average MSE: 5763359.1296
                   - Accuracy: 0.9494999999999988
Number of Trial: 9 - Average MSE: 5763363.6352
                   - Accuracy: 0.9488999999999985
Number of Trial: 10 - Average MSE: 5763363.6352
                   - Accuracy: 0.9490999999999987
[5763265.3312, 5763272.704, 57

In [2]:
# print(np.mean(accuracy_mnist))
# print(np.var(accuracy_mnist))
# plt.plot(accuracy_mnist, label = 'mean:0.95, var:2.98e-07')
# plt.legend(loc='upper right', bbox_to_anchor = (1,1.25))
# plt.xlabel('Accuracy')
# plt.ylabel('Count')
# plt.title('L2 Regularization - MNIST')
# plt.show()